In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 12.2 MB/s 
     |████████████████████████████████| 101 kB 10.9 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
     |████████████████████████████████| 6.6 MB 32.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 14.3 MB/s 


In [3]:
!pip3 install mvlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 8.6 MB/s 
     |████████████████████████████████| 38.1 MB 10.6 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
os.chdir('drive/MyDrive/William_2022/DATASETS/trans_encoder_2_KEMO')

In [32]:
!python3 DCCA_fusion.py

[ Epoch 0 ]
- (Training)  :   0% 0/35 [00:00<?, ?it/s]/content/drive/.shortcut-targets-by-id/1pzxxbb04a-cju1xepvLaBroeYAKl_mTd/William_2022/DATASETS/trans_encoder_2_KEMO/CCA.py:53: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2524.)
  [D1, V1] = torch.symeig(SigmaHat11, eigenvectors=True)
DCCA_fusion.py:61: RuntimeWarning: invalid value encountered in long_scalars
  pre_i = [cm[i, i] / total_pred[i] for i in range(class_num)]
[nan, 0.14345991561181434, 0.28571428571428

In [ ]:
dcC!python3 DCCA_eeg.py

In [ ]:
!python3 DCCA_text.py

In [ ]:
class Text_EEGDataset(Dataset):
  def __init__(self, texts, signals, labels, tokenizer, max_len):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.signals = torch.FloatTensor(signals)

  @property
  def n_insts(self):
    return len(self.labels)

  @property
  def text_len(self):
    return 32
  
  def sig_len(self):
    return 48

  def __len__(self):
    return self.n_insts

  def __getitem__(self, item):
    text = str(self.texts[item])
    label = self.labels[item]
    signal = self.signals[item]

    input_ids = [self.tokenizer.encode(text, add_special_tokens=False,max_length=MAX_LEN,pad_to_max_length=True, return_token_type_ids = False, return_attention_mask = True)]   
    input_ids = np.array(input_ids)
    input_ids = input_ids/np.linalg.norm(input_ids)
    return signal, torch.FloatTensor(input_ids).flatten(), torch.tensor(label, dtype=torch.long)

In [ ]:
train_text_eeg = Text_EEGDataset(
            texts = df_train_text[:,1:],
            labels = df_train_text[:,0],
            tokenizer = tokenizer,
            max_len = MAX_LEN,
            signals = df_train_eeg[:, 1:]
        )

In [ ]:
train_loader_text_eeg = DataLoader(dataset=train_text_eeg,
                                  batch_size=batch_size,
                                  num_workers=2
                                  )

In [ ]:
for batch in train_loader_text_eeg:
  sig, text, label = batch
  print(len(sig[0]))
  print(len(text[0]))
  # print(len(label[0]))
  break

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

48
32


In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score
def svm_classify(train_data, train_label, test_data, test_label, valid_data, valid_label, C):
    """
    trains a linear SVM on the data
    input C specifies the penalty factor of SVM
    """

    print('training SVM...')
    clf = svm.LinearSVC(C=C, dual=False)
    clf.fit(train_data, train_label.ravel())
    p = clf.predict(train_data)
    train_acc = accuracy_score(train_label, p)
    p = clf.predict(test_data)
    test_acc = accuracy_score(test_label, p)
    p = clf.predict(valid_data)
    valid_acc = accuracy_score(valid_label, p)

    return [train_acc, test_acc, valid_acc]

In [ ]:
df_train1 = np.loadtxt('arousal2_trans_baseline_DCCA_only_all_pred_train.txt')
df_train1_lbl = np.loadtxt('arousal2_trans_baseline_DCCA_only_all_label_train.txt')

df_val1 = np.loadtxt('arousal2_trans_baseline_DCCA_only_all_pred_val.txt')
df_val1_lbl = np.loadtxt('arousal2_trans_baseline_DCCA_only_all_label_val.txt')

df_test1 = np.loadtxt('arousal2_trans_baseline_DCCA_only_all_pred_test.txt')
df_test1_lbl = np.loadtxt('arousal2_trans_baseline_DCCA_only_all_label_test.txt')

df_train2 = np.loadtxt('arousal2_trans_baseline_DCCA_only_all_pred2_train.txt')
df_train2_lbl = np.loadtxt('arousal2_trans_baseline_DCCA_only_all_label2_train.txt')

df_val2 = np.loadtxt('arousal2_trans_baseline_DCCA_only_all_pred2_val.txt')
df_val2_lbl = np.loadtxt('arousal2_trans_baseline_DCCA_only_all_label2_val.txt')

df_test2 = np.loadtxt('arousal2_trans_baseline_DCCA_only_all_pred2_test.txt')
df_test2_lbl = np.loadtxt('arousal2_trans_baseline_DCCA_only_all_label2_test.txt')

In [ ]:
[train_acc, test_acc, valid_acc] = svm_classify(df_train1, df_train1_lbl, df_test1, df_test1_lbl, df_val1, df_val1_lbl, 0.01)
print('Text')
print("Accuracy on view 1 (train data) is:", train_acc * 100.0)
print("Accuracy on view 1 (validation data) is:", valid_acc * 100.0)
print("Accuracy on view 1 (test data) is:", test_acc*100.0)

[train_acc, test_acc, valid_acc] = svm_classify(df_train2, df_train2_lbl, df_test2, df_test2_lbl, df_val2, df_val2_lbl, 0.01)
print('EEG')
print("Accuracy on view 1 (train data) is:", train_acc * 100.0)
print("Accuracy on view 1 (validation data) is:", valid_acc * 100.0)
print("Accuracy on view 1 (test data) is:", test_acc*100.0)

training SVM...
Text
Accuracy on view 1 (train data) is: 33.613469156762875
Accuracy on view 1 (validation data) is: 40.40875912408759
Accuracy on view 1 (test data) is: 10.679611650485436
training SVM...
EEG
Accuracy on view 1 (train data) is: 33.55404640633843
Accuracy on view 1 (validation data) is: 19.54501216545012
Accuracy on view 1 (test data) is: 18.689320388349515


# MultiModal

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import pandas as pd
import numpy as np
from model_new import Transformer, Transformer2, Encoder
from config import *
from imblearn.over_sampling import RandomOverSampler
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import tqdm
from tqdm import tqdm
from optim_new import ScheduledOptim
from torch.optim import Adam
from sklearn.metrics import confusion_matrix
from CCA import DeepCCA, cca_loss
from scipy.stats import wasserstein_distance
from mvlearn.embed import DCCA
from mvlearn.datasets import make_gaussian_mixture
from mvlearn.plotting import crossviews_plot
from dataset_new import Fusion

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

class EEGDataset(Dataset):
    def __init__(self, signal, label):

        self._signal = torch.FloatTensor(signal)
        self._label = torch.LongTensor(label)


    @property
    def n_insts(self):
        ''' Property for dataset size '''
        return len(self._label)

    @property
    def sig_len(self):
        return self._signal.shape[1]

    def __len__(self):
        return self.n_insts

    def __getitem__(self, idx):
        return self._signal[idx], self._label[idx]

class TextDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_len):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  @property
  def n_insts(self):
    return len(self.labels)

  @property
  def text_len(self):
    return 32

  def __len__(self):
    return self.n_insts

  def __getitem__(self, item):
    text = str(self.texts[item])
    label = self.labels[item]

    encoding = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length = True,
      return_attention_mask=True
      #return_tensors='pt',
    )
    return torch.FloatTensor(encoding['input_ids']).flatten(), torch.tensor(label, dtype=torch.long)



Downloading:   0%|          | 0.00/294 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
df = pd.read_csv('df.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
onehot = OneHotEncoder()
arousal = onehot.fit_transform(df[['arousal_trans']])
angry = onehot.fit_transform(df[['angry_trans']])

In [ ]:
one_hot = pd.get_dummies(df['angry_trans'])

In [ ]:
df = df.drop('angry_trans',axis = 1)
df = df.join(one_hot)

In [ ]:
one_hot

,0,1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
1640,1,0
1641,1,0
1642,1,0
1643,1,0


In [ ]:
arousal = arousal.toarray()
angry = angry.toarray()

In [ ]:
X = df.drop([0, 1], axis = 1)
y = df[[0,1]]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 2, test_size = 0.3, shuffle = True)


In [ ]:
y_train

,0,1
200,1,0
1495,1,0
249,1,0
377,1,0
477,1,0
...,...,...
1558,1,0
1608,1,0
493,1,0
527,1,0


In [ ]:
# --- Preprocess
eeg = [emotion, 'delta0_2', 'lowAlpha0_2', 'highAlpha0_2','lowBeta0_2','highBeta0_2', 'lowGamma0_2', 'middleGamma0_2', 'theta0_2',
         'delta1_2', 'lowAlpha1_2', 'highAlpha1_2', 'lowBeta1_2', 'highBeta1_2', 'lowGamma1_2', 'middleGamma1_2', 'theta1_2',
         'delta2_2', 'lowAlpha2_2', 'highAlpha2_2', 'lowBeta2_2', 'highBeta2_2', 'lowGamma2_2', 'middleGamma2_2', 'theta2_2',
         'delta3_2', 'lowAlpha3_2', 'highAlpha3_2', 'lowBeta3_2', 'highBeta3_2', 'lowGamma3_2', 'middleGamma3_2', 'theta3_2',
         'delta4_2', 'lowAlpha4_2', 'highAlpha4_2', 'lowBeta4_2', 'highBeta4_2', 'lowGamma4_2', 'middleGamma4_2', 'theta4_2',
         'delta5_2', 'lowAlpha5_2', 'highAlpha5_2', 'lowBeta5_2', 'highBeta5_2', 'lowGamma5_2', 'middleGamma5_2', 'theta5_2']
df = pd.read_csv('df.csv')

X = df.drop([emotion], axis = 1)
y= df[[emotion]]

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 2, test_size = 0.3, shuffle = True)
ros = RandomOverSampler(random_state=2)
X_resampled_text, y_resampled_text = ros.fit_resample(X_train, y_train)

X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, random_state= 2, test_size = 0.5, shuffle = True)
df_test = pd.concat([X_test, y_test], axis = 1)
df_train = pd.concat([X_resampled_text, y_resampled_text], axis = 1)
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_val = pd.concat([X_val, y_val], axis = 1)

df_train_text = df_train[[emotion, 'new_words']]
df_train_eeg = df_train[eeg]

df_val_text = df_val[[emotion, 'new_words']]
df_val_eeg = df_val[eeg]

df_test_text = df_test[[emotion, 'new_words']]
df_test_eeg = df_test[eeg]

# --- Save CSV
df_train_text.to_csv('df_train_text.csv', header = None, index = False, index_label = False)
df_train_eeg.to_csv('df_train_eeg.csv', header = None, index = False, index_label = False)

df_val_text.to_csv('df_val_text.csv', header = None, index = False, index_label = False)
df_val_eeg.to_csv('df_val_eeg.csv', header = None, index = False, index_label=False)


df_test_text.to_csv('df_test_text.csv', header = None, index = False, index_label = False)
df_test_eeg.to_csv('df_test_eeg.csv', header = None, index = False, index_label=False)

# --- Load CSV
df_train_text = pd.read_csv('df_train_text.csv', header = None).values
df_train_eeg = pd.read_csv('df_train_eeg.csv', header = None).values

df_val_text= pd.read_csv('df_val_text.csv', header = None).values
df_val_eeg = pd.read_csv('df_val_eeg.csv', header = None).values

df_test_text= pd.read_csv('df_test_text.csv', header = None).values
df_test_eeg = pd.read_csv('df_test_eeg.csv', header = None).values

In [ ]:
from torch.utils.data.sampler import BatchSampler
class BalancedBatchSampler(BatchSampler):
    """
    BatchSampler - from a MNIST-like dataset, samples n_classes and within these classes samples n_samples.
    Returns batches of size n_classes * n_samples
    """

    def __init__(self, dataset, n_classes, n_samples):
        loader = DataLoader(dataset)
        self.labels_list = []
        for _, label in loader:
            self.labels_list.append(label)
        self.labels = torch.LongTensor(self.labels_list)
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.dataset = dataset
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < len(self.dataset):
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            indices = []
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return len(self.dataset) // self.batch_size

In [ ]:
y_text_train = y_text_train.astype('int')

In [ ]:
target = df_train_text[:, 0].astype('int')

In [ ]:
class_sample_count = np.unique(target, return_counts=True)[1]

weight = 1. / class_sample_count
samples_weight = weight[target]

In [ ]:
class_sample_count = np.unique(target, return_counts=True)[1]

weight = 1. / class_sample_count
samples_weight = weight[target]
samples_weight = torch.from_numpy(samples_weight)
samples_weigth = samples_weight.double()
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [ ]:
model1 = Transformer(device = device, d_feature=train_text.text_len, d_model=16, d_inner=32,
                            n_layers=2, n_head=4, d_k=64, d_v=64, dropout=dropout,
                            class_num=3)

In [ ]:
model2 = Transformer2(device=device, d_feature=train_eeg.sig_len, d_model=d_model, d_inner=d_inner,
                            n_layers=num_layers, n_head=num_heads, d_k=64, d_v=64, dropout=dropout, class_num=3)

In [ ]:
class Fusion(nn.Module):
  def __init__(self, model1, model2, d_feature = 40, d_model = 16, class_num = class_num):
    super(Fusion, self).__init__()
    self.model1 = model1
    self.model2 = model2
    # self.classifier = nn.Linear(4, class_num)
    self.linear1_cov = nn.Conv1d(d_feature, 1, kernel_size=1)
    self.linear1_linear = nn.Linear(d_model, class_num)
    # self.linear2_cov = nn.Conv1d(d_model, 1, kernel_size=1)
    # self.linear2_linear = nn.Linear(d_feature, class_num)

  def forward(self, x1, x2):
    x1 = self.model1(x1)
    x2 = self.model2(x2)
    x = torch.cat((x1, x2), dim = 1)
    out = self.linear1_cov(x)
    out = self.linear1_linear(out)
    return out, x1, x2, x

In [ ]:
model1 = nn.DataParallel(model1)
model2 = nn.DataParallel(model2)

In [ ]:
chkpt1 = torch.load(torchload, map_location = 'cuda')
chkpt2 = torch.load(torchload2, map_location = 'cuda')


In [ ]:
model1 = model1.to(device)
model2 = model2.to(device)

In [ ]:
use_all_singular_values = False
outdim_size = class_num

In [ ]:
model = Fusion(model1, model2).to(device)

In [ ]:
def cal_loss(pred, label, device):

    cnt_per_class = np.zeros(3)

    loss = F.cross_entropy(pred, label, reduction='sum')
    pred = pred.max(1)[1]
    n_correct = pred.eq(label).sum().item()
    cnt_per_class = [cnt_per_class[j] + pred.eq(j).sum().item() for j in range(class_num)]
    return loss, n_correct, cnt_per_class

In [ ]:
optimizer = ScheduledOptim(
            Adam(filter(lambda x: x.requires_grad, model1.parameters()),
                 betas=(0.9, 0.98), eps=1e-4), d_model, warm_steps)
        
optimizer2 = ScheduledOptim(
            Adam(filter(lambda x: x.requires_grad, model2.parameters()),
                 betas=(0.9, 0.98), eps=1e-4), d_model, warm_steps)

In [ ]:
model1.train()
all_labels = []
all_res = []
all_res2 = []
total_loss = 0
total_correct = 0
#cnt_per_class = np.zeros(class_num)


dataloader_iterator = iter(train_loader_text)

for i, data1 in enumerate(train_loader_eeg):

  try:
      data2 = next(dataloader_iterator)
  except StopIteration:
      dataloader_iterator = iter(train_loader_text)
      data2 = next(dataloader_iterator)

  sig1, label1 = map(lambda x: x.to(device), data2)
  sig2, label2 = map(lambda x: x.to(device), data1)
  print(sig1)
  print(label1)
  print()
  print(sig2)
  print(label2)

  
  optimizer.zero_grad()
  out = model1(sig1)
  print(out)
  
  all_labels.extend(label1.cpu().numpy())
  all_res.extend(out.max(1)[1].cpu().numpy())
  loss, n_correct1, _ = cal_loss(out, label1, device)
  # print(loss)
  
  
  
  loss.backward()
  optimizer.step_and_update_lr()
  total_loss += loss.item()
  total_correct += (n_correct1)


  cm = confusion_matrix(all_labels, all_res)
  

# train_loss = total_loss / (total_num + total_num2)
# train_acc = total_correct / (total_num + total_num2)



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest se

tensor([[0.0000e+00, 4.8759e+04, 1.2465e+04, 3.1300e+02, 8.0000e+00, 1.7200e+02,
         6.7000e+01, 5.0000e+00, 1.2190e+03, 6.9600e+02, 4.4403e+04, 2.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00],
        [0.0000e+00, 4.8759e+04, 4.3210e+03, 1.4775e+04, 7.1180e+03, 5.0000e+01,
         1.3000e+01, 1.2000e+02, 1.3000e+01, 1.3110e+03, 4.9000e+01, 4.4403e+04,
         2.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00],
        [0.0000e+00, 4.9329e+04, 4.6300e+02, 3.8000e+01, 3.9190e+03, 6.8250e+03,
         3.8000e+01, 4.3700e+02, 1.4000e+

KeyboardInterrupt: ignored

In [ ]:
'''
H1 = text
H2 = eeg

sig1 = text
sig2 = eeg

o1 = o2 = size of text

m = text row length

H1bar = at each row take the mean
'''
r1= 1e-3
r2 = 1e-3
eps = 1e-9

H1, H2 = sig1.t(), sig2.t()

o1 = H1.size(0)
o2 = H2.size(0)
m = H1.size(1)

H1bar = H1 - H1.mean(dim = 1).unsqueeze(dim=1)
H2bar = H2 - H2.mean(dim =1).unsqueeze(dim=1)

SigmaHat12 = (1.0/(m-1)) * torch.matmul(H1bar, H2bar.t())
SigmaHat11 = (1.0 / (m-1)) * torch.matmul(H1bar, H1bar.t()) + r1 * torch.eye(o1, device = device)
SigmaHat22 = (1.0 / (m-1)) * torch.matmul(H2bar, H2bar.t()) + r2 * torch.eye(o2, device = device)


[D1, V1] = torch.symeig(SigmaHat11, eigenvectors = True)
[D2, V2] = torch.symeig(SigmaHat22, eigenvectors = True)

posInd1 = torch.gt(D1, eps).nonzero()[:, 0]
D1 = D1[posInd1]
V1 = V1[:, posInd1]
posInd2 = torch.gt(D2, eps).nonzero()[:, 0]
D2 = D2[posInd2]
V2 = V2[:, posInd2]



SigmaHat11RootInv = torch.matmul(
    torch.matmul(V1, torch.diag(D1 ** -0.5)), V1.t())
SigmaHat22RootInv = torch.matmul(
    torch.matmul(V2, torch.diag(D2 ** -0.5)), V2.t())

Tval = torch.matmul(torch.matmul(SigmaHat11RootInv,
                                  SigmaHat12), SigmaHat22RootInv)


trace_TT = torch.matmul(Tval.t(), Tval)
trace_TT = torch.add(trace_TT, (torch.eye(trace_TT.shape[0])*r1).to(device)) # regularization for more stability
U, V = torch.symeig(trace_TT, eigenvectors=True)
U = torch.where(U>eps, U, (torch.ones(U.shape).float()*eps).to(device))
U = U.topk(outdim_size)[0]
corr = torch.sum(torch.sqrt(U))

In [ ]:
corr

tensor(1.9993)